# 📓 TruLens Quickstart

In this quickstart you will create a RAG from scratch and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/quickstart/quickstart.ipynb)

In [22]:
!pip uninstall --pre trulens trulens-providers-openai 


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

no such option: --pre


In [1]:
!pip install openai


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/cf/9e/d77569d06e365f093977d94f305a395b7ac5ccd746016a2e8dd34c4e20c1/openai-1.42.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/7b/a2/10639a79341f6c019dedc95bd48a4928eed9f1d1197f4c04f546fc7ae0ff/anyio-4.4.0-py3-none-any.whl.metadata
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 fr

In [1]:
import openai 
from openai import AzureOpenAI
azure_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQ0MjU4MzYsIm5iZiI6MTcyNDQyNTgzNiwiZXhwIjoxNzI0NDMwOTA4LCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBRzB2R3VMY3poTFEvZTFqMk5KbERDbmJkMndObjNVc3NvYVVZdDRDeFk5RlFmRTdWcDBRQ005a3BDSW9Yb01zS3doeGFNbVhsYWRrd3ZrVGhlREdZbUx0aWxodGVaNngwbkhTSGpheC9mUG1FTkhiK2ZrUVlBTWlZUWRMaHRFNXIiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMiIsIm5hbWUiOiJTdWJocmFuZWVsIENoYWtyYWJvcnR5Iiwib2lkIjoiOGM4ZWRhM2UtZDEyMC00YzE4LTg0YTItMTBmODAyYTg1OGJhIiwib25wcmVtX3NpZCI6IlMtMS01LTIxLTI1NzQyMTg5NDQtMzEyNjcxOTQ2NC0xMjczNzA0NDcwLTk1NzQwIiwicHVpZCI6IjEwMDMyMDAzMzJCQ0MyQUMiLCJyaCI6IjAuQVRjQUF3Z1RPVGdLd1V5aEZCRjc1eVZjN1pBaU1YM0lLRHhIb08yT1UzU2JiVzAzQUVrLiIsInNjcCI6InVzZXJfaW1wZXJzb25hdGlvbiIsInN1YiI6Ild1b1hQX3dmb0MxNDJYakNFYXVYeHY1eUw2SlFqQ25BVjlmdk9CYzZYWlUiLCJ0aWQiOiIzOTEzMDgwMy0wYTM4LTRjYzEtYTExNC0xMTdiZTcyNTVjZWQiLCJ1bmlxdWVfbmFtZSI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXBuIjoiY2hha3N1YkBhY3VpdHlrcC5jb20iLCJ1dGkiOiJ4aXZyeTNVYjkwNnp0aUZINWFWMkFBIiwidmVyIjoiMS4wIiwid2lkcyI6WyJiNzlmYmY0ZC0zZWY5LTQ2ODktODE0My03NmIxOTRlODU1MDkiXSwieG1zX2lkcmVsIjoiMSA2In0.KCCLgD-aan3wzFWpCAqVgNqtZotTbwJYwrJmZnc42Cj61DnfAhf_uKWpqQKfMkqUj_0HCx14eZU7SkssehD5xH4O2Cm_1fLEEMyLU98iPpBNjREcp5WZwWqljAvVIPZR2KHZxuqOldIl3RJDDo_HkpIaQ82vjMZVoiiTjmDvE-Fp1Ud99eduFqh4RcjqQwUwGKEAIBY1xBBdgoYDUerWLTh4n3fWdoMn6rzTozS_4grWS2PAMpKHl6_1V55YU6Szt5oZ-oPd8KadqLE2wubDV7RK0FFJt17SLdBg5iXa2Be8BRAr-bILWbnL-Yt-K2Ez1sp8UWhKUWw0XLPYiuwr5w"
#"eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQ0MjA1MjAsIm5iZiI6MTcyNDQyMDUyMCwiZXhwIjoxNzI0NDI2MTYxLCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBdHFjQkhLSDc2RysrTWVPVk9xbHN4SFU3ZGh2L0JYdW5RT2RTcE9uMDR5NzdzUzIyT25xU2Q3NUdCRTB4cDc2UUQ0MXlSSkwzbFlyUWZCNWx1bXBwbzhzL3VkUDZVSnJVUGo5Yko1eE1PQkdtbnZZYlJpeTl0WlY5a25Md0VRYUUiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMTkiLCJuYW1lIjoiU3ViaHJhbmVlbCBDaGFrcmFib3J0eSIsIm9pZCI6IjhjOGVkYTNlLWQxMjAtNGMxOC04NGEyLTEwZjgwMmE4NThiYSIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0yNTc0MjE4OTQ0LTMxMjY3MTk0NjQtMTI3MzcwNDQ3MC05NTc0MCIsInB1aWQiOiIxMDAzMjAwMzMyQkNDMkFDIiwicmgiOiIwLkFUY0FBd2dUT1RnS3dVeWhGQkY3NXlWYzdaQWlNWDNJS0R4SG9PMk9VM1NiYlcwM0FFay4iLCJzY3AiOiJ1c2VyX2ltcGVyc29uYXRpb24iLCJzdWIiOiJXdW9YUF93Zm9DMTQyWGpDRWF1WHh2NXlMNkpRakNuQVY5ZnZPQmM2WFpVIiwidGlkIjoiMzkxMzA4MDMtMGEzOC00Y2MxLWExMTQtMTE3YmU3MjU1Y2VkIiwidW5pcXVlX25hbWUiOiJjaGFrc3ViQGFjdWl0eWtwLmNvbSIsInVwbiI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXRpIjoiSWZISUp1NUt0RUtWRWJmWmkxQ3FBQSIsInZlciI6IjEuMCIsIndpZHMiOlsiYjc5ZmJmNGQtM2VmOS00Njg5LTgxNDMtNzZiMTk0ZTg1NTA5Il0sInhtc19pZHJlbCI6IjEgMjgifQ.q8ygUZArPO9Z9Od3pGpdLgw7uGFdyntcz3AmxakDMiEFz4m7_prk4a6af6xHWQOOp-SiZH6PW_ExJ34WgtjxcgcVc6T6wzHINZK6FltMu6bUXYVPsQenSCSTYFwqMZxXQxjN2ZO3igMrBDMGNhyAAb0n0MRE0uv6WZtYuM02_nNN6UGhzcyUJwSInHPwCnJ_mXLe8IVwqt6ilHfZLQWsIzrw0_hzF0pMQA_4ZulEjT2x9l_AjZgTCDbZ9A5xhRbmABVUwxYMpfKEYmIoUESab3HOFY1VX0Q-nXQFWT_lJcXzP0m-eyZRHsuZ25gBXPL5SEGtPTSuLXZa1goMzpm_8Q"

api_version = "2023-05-15"
endpoint = "https://beat-sandbox-openai-00001.openai.azure.com/"
  
client = AzureOpenAI(
    azure_deployment='text-embedding-ada-002',
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token=azure_token,
)



In [2]:
import openai
from openai import AzureOpenAI

# Initialize the AzureOpenAI client
azure_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQ0MjU4MzYsIm5iZiI6MTcyNDQyNTgzNiwiZXhwIjoxNzI0NDMwOTA4LCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBRzB2R3VMY3poTFEvZTFqMk5KbERDbmJkMndObjNVc3NvYVVZdDRDeFk5RlFmRTdWcDBRQ005a3BDSW9Yb01zS3doeGFNbVhsYWRrd3ZrVGhlREdZbUx0aWxodGVaNngwbkhTSGpheC9mUG1FTkhiK2ZrUVlBTWlZUWRMaHRFNXIiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMiIsIm5hbWUiOiJTdWJocmFuZWVsIENoYWtyYWJvcnR5Iiwib2lkIjoiOGM4ZWRhM2UtZDEyMC00YzE4LTg0YTItMTBmODAyYTg1OGJhIiwib25wcmVtX3NpZCI6IlMtMS01LTIxLTI1NzQyMTg5NDQtMzEyNjcxOTQ2NC0xMjczNzA0NDcwLTk1NzQwIiwicHVpZCI6IjEwMDMyMDAzMzJCQ0MyQUMiLCJyaCI6IjAuQVRjQUF3Z1RPVGdLd1V5aEZCRjc1eVZjN1pBaU1YM0lLRHhIb08yT1UzU2JiVzAzQUVrLiIsInNjcCI6InVzZXJfaW1wZXJzb25hdGlvbiIsInN1YiI6Ild1b1hQX3dmb0MxNDJYakNFYXVYeHY1eUw2SlFqQ25BVjlmdk9CYzZYWlUiLCJ0aWQiOiIzOTEzMDgwMy0wYTM4LTRjYzEtYTExNC0xMTdiZTcyNTVjZWQiLCJ1bmlxdWVfbmFtZSI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXBuIjoiY2hha3N1YkBhY3VpdHlrcC5jb20iLCJ1dGkiOiJ4aXZyeTNVYjkwNnp0aUZINWFWMkFBIiwidmVyIjoiMS4wIiwid2lkcyI6WyJiNzlmYmY0ZC0zZWY5LTQ2ODktODE0My03NmIxOTRlODU1MDkiXSwieG1zX2lkcmVsIjoiMSA2In0.KCCLgD-aan3wzFWpCAqVgNqtZotTbwJYwrJmZnc42Cj61DnfAhf_uKWpqQKfMkqUj_0HCx14eZU7SkssehD5xH4O2Cm_1fLEEMyLU98iPpBNjREcp5WZwWqljAvVIPZR2KHZxuqOldIl3RJDDo_HkpIaQ82vjMZVoiiTjmDvE-Fp1Ud99eduFqh4RcjqQwUwGKEAIBY1xBBdgoYDUerWLTh4n3fWdoMn6rzTozS_4grWS2PAMpKHl6_1V55YU6Szt5oZ-oPd8KadqLE2wubDV7RK0FFJt17SLdBg5iXa2Be8BRAr-bILWbnL-Yt-K2Ez1sp8UWhKUWw0XLPYiuwr5w"
api_version = "2023-05-15"
endpoint = "https://beat-sandbox-openai-00001.openai.azure.com/"

client = AzureOpenAI(
    azure_deployment='text-embedding-ada-002',
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token=azure_token,
)

# Define a sample text
sample_text = "Hello, this is a test to generate embeddings using Azure OpenAI."

# Generate embeddings
response = client.embeddings.create(input=sample_text, model="text-embedding-ada-002")

# Access and print the embeddings
embeddings = response.data[0].embedding
print(embeddings)

[-0.029876357, -0.0113770235, -0.011550413, -0.010363362, 0.013037562, 0.03299737, -0.018419309, -0.013484374, -0.02216719, -0.03806568, 0.015738439, 0.022033814, -0.0051016547, -0.006748856, -0.0027708989, 0.0042080316, 0.0018622708, -0.009089615, 0.013844491, -0.009883205, -0.023807723, -0.015351647, 0.009042934, 0.0064487583, -0.021593671, -0.01892614, 0.012964205, -0.036491837, 0.01504488, -0.029796332, 0.01976641, -0.0013737789, -0.0012570744, -0.025661657, -0.0019973146, -0.0029509573, 0.008342707, -0.0011770484, 0.02411449, -0.011590426, 0.0008002596, 0.001005326, 0.010243323, -0.022607334, -0.018379295, 0.006735518, -0.0023841066, -0.0048315674, 0.014418011, 0.027688982, 0.018699398, 0.005881908, -0.030516565, -0.007982589, -0.003961285, 0.0037112038, -0.022487294, 0.027715657, 0.009276343, -0.005771872, 0.001205391, -0.0019372951, -0.0038645868, 0.0066188136, -0.012770809, -0.0013095915, 0.011096933, 0.003351087, 0.02439458, -0.0060686353, 0.023380918, 0.008462745, 0.000613115

## Get Data

In this case, we'll just initialize some simple text in the notebook.

## Create Vector Store

Create a opensearch vector store in memory.

In [4]:
!pip install opensearch-py


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from opensearchpy import OpenSearch
import ssl
import certifi

# Create an SSL context
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Initialize the OpenSearch client with the SSL context
direct_os_client = OpenSearch(
    hosts=[{"host": "localhost", "port": 9200}],
    http_auth=("admin", "H@thork!7210"),
    use_ssl=True,
    verify_certs=True,  # Enable certificate verification
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    ssl_context=ssl_context
)

c:\Users\chaksub\AppData\Local\Programs\Python\Python311\Lib\site-packages\opensearchpy\connection\http_urllib3.py:163: UserWarning: When using `ssl_context`, all other SSL related kwargs are ignored
  warnings.warn(


Populate the vector store.

In [4]:
import re
import time
from tqdm import trange
from opensearchpy import OpenSearch
import openai

# Define the texts
uw_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

wsu_info = """
Washington State University, commonly known as WSU, founded in 1890, is a public research university in Pullman, Washington.
With multiple campuses across the state, it is the state's second largest institution of higher education.
WSU is known for its programs in veterinary medicine, agriculture, engineering, architecture, and pharmacy.
"""

seattle_info = """
Seattle, a city on Puget Sound in the Pacific Northwest, is surrounded by water, mountains and evergreen forests, and contains thousands of acres of parkland.
It's home to a large tech industry, with Microsoft and Amazon headquartered in its metropolitan area.
The futuristic Space Needle, a legacy of the 1962 World's Fair, is its most iconic landmark.
"""

starbucks_info = """
Starbucks Corporation is an American multinational chain of coffeehouses and roastery reserves headquartered in Seattle, Washington.
As the world's largest coffeehouse chain, Starbucks is seen to be the main representation of the United States' second wave of coffee culture.
"""

# Combine all texts into a list
texts = [uw_info, wsu_info, seattle_info, starbucks_info]

# Split texts into chunks (assuming each chunk is a sentence for simplicity)
splitted_chunks = [sentence for text in texts for sentence in text.split('\n') if sentence]

# Initialize OpenSearch client
os_client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', 'admin'),
    use_ssl=False,
    verify_certs=False,
    ssl_show_warn=False,
)

# Create index if it doesn't exist
index_name = "embedded_texts"
if not os_client.indices.exists(index=index_name):
    os_client.indices.create(index=index_name)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

# Set OpenAI API key
openai.api_key = azure_token

def create_embeddings(text):
    try:
        response = client.embeddings.create(
            input=[text],
            model="text-embedding-ada-002"
        )
        embeddings = response.data[0].embedding 
        print(f"Created embedding for text: {text[:30]}...")  # Log the text and embedding creation
        return embeddings
    except Exception as e:
        print(f"Error creating embedding for text: {text[:30]}... Error: {e}")
        raise

# Embed and save chunks to OpenSearch
for i in trange(0, len(splitted_chunks), 100):
    batch = splitted_chunks[i: i + 100]
    count = 3
    while count > 0:
        try:
            for j, chunk in enumerate(batch):
                embedding = create_embeddings(chunk)
                document = {
                    "text_id": f"chunk_{i + j}",
                    "text": chunk,
                    "embedding": embedding
                }
                response = os_client.index(index=index_name, body=document)
                print(f"Indexed document: {document['text_id']}, Response: {response['result']}")
            time.sleep(0.1)
        except Exception as err:
            print(f'Additional retrying {(3 - count) + 1} for embedding... Error: {err}')
            t = re.search(r'(\d+)\.?.?.?second', str(err))
            if t:
                t = int(t.groups()[0]) + 1
                time.sleep(t)
                print(f'Additional waiting for {t} seconds')
        else:
            break

        count -= 1

# Verify and print documents in the index
def print_documents(index_name):
    try:
        response = os_client.search(index=index_name, body={"query": {"match_all": {}}})
        hits = response['hits']['hits']
        print(f"Total documents found: {len(hits)}")
        for hit in hits:
            print(hit['_source'])
    except Exception as e:
        print("Error retrieving documents:", e)

print_documents(index_name)

Index 'embedded_texts' already exists.


  0%|          | 0/1 [00:00<?, ?it/s]

Created embedding for text: The University of Washington, ...
Indexed document: chunk_0, Response: created
Created embedding for text: with over 45,000 students acro...
Indexed document: chunk_1, Response: created
Created embedding for text: As the flagship institution of...
Indexed document: chunk_2, Response: created
Created embedding for text: UW encompasses over 500 buildi...
Indexed document: chunk_3, Response: created
Created embedding for text: including one of the largest l...
Indexed document: chunk_4, Response: created
Created embedding for text: Washington State University, c...
Indexed document: chunk_5, Response: created
Created embedding for text: With multiple campuses across ...
Indexed document: chunk_6, Response: created
Created embedding for text: WSU is known for its programs ...
Indexed document: chunk_7, Response: created
Created embedding for text: Seattle, a city on Puget Sound...
Indexed document: chunk_8, Response: created
Created embedding for text: It's home

100%|██████████| 1/1 [00:04<00:00,  4.77s/it]

Created embedding for text: As the world's largest coffeeh...
Indexed document: chunk_12, Response: created


Total documents found: 10
{'text_id': 'chunk_0', 'text': 'The University of Washington, founded in 1861 in Seattle, is a public research university', 'embedding': [0.004950651, -0.00999016, -0.016641805, -0.017314585, -0.023293449, -0.0046523428, -0.0052203983, 0.0044206777, -0.008092411, -0.022252543, -0.022544503, 0.00762908, 0.03437529, -0.015880166, -0.0009695025, -0.030008562, 0.013354064, -0.040290684, -0.0032226834, -0.015385101, -0.017949283, -0.009634729, 0.0033416895, -0.010802574, -0.00933642, 0.018825168, -0.001161499, -0.020640407, 0.029018432, -0.00887309, -0.0045285765, -0.0052743475, -0.04392116, -0.013493698, -0.014953505, -0.004236615, -0.013354064, 0.005429849, 0.0039097453, -0.0055885236, 0.011881563, -0.000360985, -0.00288788, -0.009844179, -0.016260985, 0.014306112, -0.0014193454, -0.027520543, -0.0070578516, 0.0062263957, 0.017124176, 0.012179871, -0.015550123, -0.016007105, -0.017466912, -0.028459897, -0.008796927, 0.018583983, -0.023153815, -0.009082541, -0.004

In [7]:
# from openai import AzureOpenAI
 
# api_version = "2023-05-15"
# endpoint = "https://beat-sandbox-openai-00001.openai.azure.com/"
 
# client = AzureOpenAI(
#     azure_deployment='gpt-35-turbo-16k',
#     api_version=api_version,
#     azure_endpoint=endpoint,
#     azure_ad_token=azure_token,
# )
# client.chat.completions.create(
#             model="gpt-35-turbo-16k",
#             messages=[
#                 {"role": "system", "content": "You are a helpful assistant."},
#                 {"role": "user", "content": "Hello, world!"}
#             ]
#         )

In [5]:


from opensearchpy import OpenSearch

# Initialize OpenSearch client without SSL
os_client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', 'H@thork!7210'),
    use_ssl=False,  # Disable SSL
    verify_certs=False  # Disable certificate verification
)

# Get the list of all indices
try:
    indices = os_client.cat.indices(format='json')
    print(f"Total number of indices: {len(indices)}")
    for index in indices:
        print(index['index'])
except Exception as e:
    print("Error retrieving indices:", e)

Total number of indices: 3
.plugins-ml-config
.opensearch-observability
embedded_texts


In [6]:
from opensearchpy import OpenSearch

# Initialize OpenSearch client without SSL
os_client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', 'H@thork!7210'),
    use_ssl=False,  # Disable SSL
    verify_certs=False  # Disable certificate verification
)

# Define the index name
index_name = 'embedded_texts'

# Check if the index exists
try:
    if not os_client.indices.exists(index=index_name):
        print(f"Index '{index_name}' does not exist.")
    else:
        print(f"Index '{index_name}' exists.")

        # Check the document count in the index
        doc_count = os_client.count(index=index_name)['count']
        print(f"Document count in '{index_name}': {doc_count}")

        # Retrieve a document from the index
        response = os_client.search(index=index_name, body={"query": {"match_all": {}}}, size=1)
        
        # Check if any documents are returned
        if response['hits']['total']['value'] > 0:
            # Get the first document
            doc = response['hits']['hits'][0]['_source']
            
            # Print the embedding field
            if 'embedding' in doc:
                print("Embedding field found in the document:")
                print(doc['embedding'])
            else:
                print("Embedding field not found in the document.")
        else:
            print("No documents found in the index.")
except Exception as e:
    print("Error:", e)

# Get the list of all indices
try:
    indices = os_client.cat.indices(format='json')
    print(f"Total number of indices: {len(indices)}")
    for index in indices:
        print(index['index'])
except Exception as e:
    print("Error retrieving indices:", e)

Index 'embedded_texts' exists.
Document count in 'embedded_texts': 39
Embedding field found in the document:
[0.004950651, -0.00999016, -0.016641805, -0.017314585, -0.023293449, -0.0046523428, -0.0052203983, 0.0044206777, -0.008092411, -0.022252543, -0.022544503, 0.00762908, 0.03437529, -0.015880166, -0.0009695025, -0.030008562, 0.013354064, -0.040290684, -0.0032226834, -0.015385101, -0.017949283, -0.009634729, 0.0033416895, -0.010802574, -0.00933642, 0.018825168, -0.001161499, -0.020640407, 0.029018432, -0.00887309, -0.0045285765, -0.0052743475, -0.04392116, -0.013493698, -0.014953505, -0.004236615, -0.013354064, 0.005429849, 0.0039097453, -0.0055885236, 0.011881563, -0.000360985, -0.00288788, -0.009844179, -0.016260985, 0.014306112, -0.0014193454, -0.027520543, -0.0070578516, 0.0062263957, 0.017124176, 0.012179871, -0.015550123, -0.016007105, -0.017466912, -0.028459897, -0.008796927, 0.018583983, -0.023153815, -0.009082541, -0.004512709, -0.005801148, -0.021592455, -0.011240517, -0.0

In [7]:
# from opensearchpy import OpenSearch

# # Initialize OpenSearch client without SSL
# os_client = OpenSearch(
#     hosts=[{'host': 'localhost', 'port': 9200}],
#     http_auth=('admin', 'H@thork!7210'),
#     use_ssl=False,  # Disable SSL
#     verify_certs=False  # Disable certificate verification
# )

# # Function to delete all indices
# def delete_all_indices(client):
#     try:
#         # Get the list of all indices
#         indices = client.cat.indices(format='json')
#         print(f"Total number of indices: {len(indices)}")

#         # Iterate over the list of indices and delete each one
#         for index in indices:
#             index_name = index['index']
#             print(f"Deleting index: {index_name}")
#             client.indices.delete(index=index_name)
#         print("All indices have been deleted.")
#     except Exception as e:
#         print("Error deleting indices:", e)

# # Call the function to delete all indices
# delete_all_indices(os_client)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [11]:
!pip install trulens


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
from trulens.core import Tru
from trulens.core.app.custom import instrument

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [11]:
# azure_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQ0MjA1MjAsIm5iZiI6MTcyNDQyMDUyMCwiZXhwIjoxNzI0NDI2MTYxLCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBdHFjQkhLSDc2RysrTWVPVk9xbHN4SFU3ZGh2L0JYdW5RT2RTcE9uMDR5NzdzUzIyT25xU2Q3NUdCRTB4cDc2UUQ0MXlSSkwzbFlyUWZCNWx1bXBwbzhzL3VkUDZVSnJVUGo5Yko1eE1PQkdtbnZZYlJpeTl0WlY5a25Md0VRYUUiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMTkiLCJuYW1lIjoiU3ViaHJhbmVlbCBDaGFrcmFib3J0eSIsIm9pZCI6IjhjOGVkYTNlLWQxMjAtNGMxOC04NGEyLTEwZjgwMmE4NThiYSIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0yNTc0MjE4OTQ0LTMxMjY3MTk0NjQtMTI3MzcwNDQ3MC05NTc0MCIsInB1aWQiOiIxMDAzMjAwMzMyQkNDMkFDIiwicmgiOiIwLkFUY0FBd2dUT1RnS3dVeWhGQkY3NXlWYzdaQWlNWDNJS0R4SG9PMk9VM1NiYlcwM0FFay4iLCJzY3AiOiJ1c2VyX2ltcGVyc29uYXRpb24iLCJzdWIiOiJXdW9YUF93Zm9DMTQyWGpDRWF1WHh2NXlMNkpRakNuQVY5ZnZPQmM2WFpVIiwidGlkIjoiMzkxMzA4MDMtMGEzOC00Y2MxLWExMTQtMTE3YmU3MjU1Y2VkIiwidW5pcXVlX25hbWUiOiJjaGFrc3ViQGFjdWl0eWtwLmNvbSIsInVwbiI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXRpIjoiSWZISUp1NUt0RUtWRWJmWmkxQ3FBQSIsInZlciI6IjEuMCIsIndpZHMiOlsiYjc5ZmJmNGQtM2VmOS00Njg5LTgxNDMtNzZiMTk0ZTg1NTA5Il0sInhtc19pZHJlbCI6IjEgMjgifQ.q8ygUZArPO9Z9Od3pGpdLgw7uGFdyntcz3AmxakDMiEFz4m7_prk4a6af6xHWQOOp-SiZH6PW_ExJ34WgtjxcgcVc6T6wzHINZK6FltMu6bUXYVPsQenSCSTYFwqMZxXQxjN2ZO3igMrBDMGNhyAAb0n0MRE0uv6WZtYuM02_nNN6UGhzcyUJwSInHPwCnJ_mXLe8IVwqt6ilHfZLQWsIzrw0_hzF0pMQA_4ZulEjT2x9l_AjZgTCDbZ9A5xhRbmABVUwxYMpfKEYmIoUESab3HOFY1VX0Q-nXQFWT_lJcXzP0m-eyZRHsuZ25gBXPL5SEGtPTSuLXZa1goMzpm_8Q"

# #"eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQ0MDIwNTUsIm5iZiI6MTcyNDQwMjA1NSwiZXhwIjoxNzI0NDA3MDcwLCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBSXpJYVpodTNPSTJRSU91VnQ3Ym5JbytuUnYvSHVLQWVNdytsNjhQSHRHU0MzbmtCazQ2YitJYW5CNlQ1SVZWSmxzcW1SUGkzRGpIOFZES2U0QVdhb1IzUVdjczlseExhc2svQStlTCtzOEJZM2ltR0NSeGlnL2FoSm0xb3BCdFAiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMTkiLCJuYW1lIjoiU3ViaHJhbmVlbCBDaGFrcmFib3J0eSIsIm9pZCI6IjhjOGVkYTNlLWQxMjAtNGMxOC04NGEyLTEwZjgwMmE4NThiYSIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0yNTc0MjE4OTQ0LTMxMjY3MTk0NjQtMTI3MzcwNDQ3MC05NTc0MCIsInB1aWQiOiIxMDAzMjAwMzMyQkNDMkFDIiwicmgiOiIwLkFUY0FBd2dUT1RnS3dVeWhGQkY3NXlWYzdaQWlNWDNJS0R4SG9PMk9VM1NiYlcwM0FFay4iLCJzY3AiOiJ1c2VyX2ltcGVyc29uYXRpb24iLCJzdWIiOiJXdW9YUF93Zm9DMTQyWGpDRWF1WHh2NXlMNkpRakNuQVY5ZnZPQmM2WFpVIiwidGlkIjoiMzkxMzA4MDMtMGEzOC00Y2MxLWExMTQtMTE3YmU3MjU1Y2VkIiwidW5pcXVlX25hbWUiOiJjaGFrc3ViQGFjdWl0eWtwLmNvbSIsInVwbiI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXRpIjoiTk4ybGF3bmwyMEM1eENPU2t6NHNBQSIsInZlciI6IjEuMCIsIndpZHMiOlsiYjc5ZmJmNGQtM2VmOS00Njg5LTgxNDMtNzZiMTk0ZTg1NTA5Il0sInhtc19pZHJlbCI6IjYgMSJ9.ENPcQ7M4j9Pk14fE4xnTAaLZ_letD5aF8l6q-WN_19EzTRVjsdNTK3PT3a8LDuJsnOc-bnrklfIKy4QKosANnytYBWufXoLHQ27i3lomOIfj9Bl4RkdnXCoXXsm1_TMgB-EkiUF0wcmZCWrH9e9S4zXA8iNLklaA0P4QQUxLgE55sLl0Asz3dJ8dMQ7bN_TCu97zBdyXGBNCcvaVG7DPILVTcfpPOs_om26qHmvKGur03Qf-eY1pJucKRYCon66eTmfsRnSPEy-RPKOR47MNmbphtXjWGhQOdfZ4qFdyc9IfhAiM8KJXHXUkoqhDdRBXbsf_dgCLumVCAVXKixMoTQ"
# #"eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQzOTU1MzYsIm5iZiI6MTcyNDM5NTUzNiwiZXhwIjoxNzI0NDAwODI1LCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBc2xlWUhMUUhZZzZwb25HeWpSM0lFb29TRWFXMDFmWk1LejkrS2ZxcEdUUVgxOEZNMU9BWnRlYy9TbndPaUtqUDF2K3BsUDdnTTZhSkV3NmlkQ3RGVy9XdnQrNVhyRURQeVk3WFJYOTNwbFlnd0lZcUFlalFqa0o1elRtZllDeXAiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMTkiLCJuYW1lIjoiU3ViaHJhbmVlbCBDaGFrcmFib3J0eSIsIm9pZCI6IjhjOGVkYTNlLWQxMjAtNGMxOC04NGEyLTEwZjgwMmE4NThiYSIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0yNTc0MjE4OTQ0LTMxMjY3MTk0NjQtMTI3MzcwNDQ3MC05NTc0MCIsInB1aWQiOiIxMDAzMjAwMzMyQkNDMkFDIiwicmgiOiIwLkFUY0FBd2dUT1RnS3dVeWhGQkY3NXlWYzdaQWlNWDNJS0R4SG9PMk9VM1NiYlcwM0FFay4iLCJzY3AiOiJ1c2VyX2ltcGVyc29uYXRpb24iLCJzdWIiOiJXdW9YUF93Zm9DMTQyWGpDRWF1WHh2NXlMNkpRakNuQVY5ZnZPQmM2WFpVIiwidGlkIjoiMzkxMzA4MDMtMGEzOC00Y2MxLWExMTQtMTE3YmU3MjU1Y2VkIiwidW5pcXVlX25hbWUiOiJjaGFrc3ViQGFjdWl0eWtwLmNvbSIsInVwbiI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXRpIjoiU3VYMjE2U2RzRXFlZUh4TFZuWnVBQSIsInZlciI6IjEuMCIsIndpZHMiOlsiYjc5ZmJmNGQtM2VmOS00Njg5LTgxNDMtNzZiMTk0ZTg1NTA5Il0sInhtc19pZHJlbCI6IjggMSJ9.Jbhy70zWlB8960YvfhjFGuCKFzSAjTPNxeBYGGPdTF5hVdP3FOdGRaSKQZiY4F_Esg4voQvSme2DYzkt6ippD5rQOvXi81k1SOJyqk61vvK38LWVT9yWfAntPC2eJpY64y2yPDW7jcuq69GqqtFCxbuFr6Mb5fD2ygqOU0Bjjd0sTK_ju7AMyT64YX0kt0_jGHeqkLjun_4EUkSFs5zav446RpiAOySkSSwhcvyZXCDC4F61duXrmDKMEOiBTKU9OAqjvFGRM-k4lfRA9nsBKn8GXAtkJC36xTdaKrvL9hi8PnN19lOT0g4eLJaLE6HcmFuBSEJK9ixaKdy-Tljiog"
# #"eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCIsImtpZCI6IktRMnRBY3JFN2xCYVZWR0JtYzVGb2JnZEpvNCJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzM5MTMwODAzLTBhMzgtNGNjMS1hMTE0LTExN2JlNzI1NWNlZC8iLCJpYXQiOjE3MjQzOTU1MzYsIm5iZiI6MTcyNDM5NTUzNiwiZXhwIjoxNzI0NDAwODI1LCJhY3IiOiIxIiwiYWlvIjoiQVdRQW0vOFhBQUFBc2xlWUhMUUhZZzZwb25HeWpSM0lFb29TRWFXMDFmWk1LejkrS2ZxcEdUUVgxOEZNMU9BWnRlYy9TbndPaUtqUDF2K3BsUDdnTTZhSkV3NmlkQ3RGVy9XdnQrNVhyRURQeVk3WFJYOTNwbFlnd0lZcUFlalFqa0o1elRtZllDeXAiLCJhbXIiOlsicHdkIiwicnNhIiwibWZhIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZGV2aWNlaWQiOiJmMmViMDUzOS02OGY2LTQwY2ItOTllMy03MTkzYTIwYTZmNzciLCJmYW1pbHlfbmFtZSI6IkNoYWtyYWJvcnR5IiwiZ2l2ZW5fbmFtZSI6IlN1YmhyYW5lZWwiLCJncm91cHMiOlsiNzMyZjgxMGYtYmJhYS00MjBmLThiYjgtYzc5OTllYmVjYmQzIiwiNzQ1YWM4MWEtM2RmNy00OWM1LWE2ODQtNDQwY2NmN2FiYTE5IiwiZDZlZjk1MjUtYjM3My00MzY2LWIwZTMtOTE3YzAwOTlmMjU0IiwiYzk4NjdlMzMtZTU0My00OWU5LTg4NWUtNGU0ZmRmNDg4YjI4IiwiODRjNTkyNDAtZjQzNi00MjE5LTljOWUtMmMyYmMzMjkzM2UwIiwiY2NmNGVhNDgtOWY4Yy00ZDYzLWI4MzgtMjY2MDM5OThlZmNmIiwiZWI3OWRhNGEtMGY3NS00YzBjLWIxYWYtNzNjOTE4OTRlNzg3IiwiMTU2MzM1NTItZDI5OC00ZTI2LWE1Y2EtMzFjMjkzNDlkZjUwIiwiMGRmNWI3NWQtNWE1Yy00ZWZhLTk2OTItMThjZjM1YTcwNmMyIiwiMmRhOTA0NjYtM2U1Zi00NzkyLWJjYWEtYjE4N2ZlNWMzMTgzIiwiYzVmMWNhNzMtZWU1ZC00MmZlLWIxYTMtOTk2MTYzMzBjYTFkIiwiMDViYjQyNzktMWZlMS00OTdhLWEyMDktNmM1YjdlMDcxYzg4IiwiNmM3NTI2N2YtNmU1Ni00Zjc3LTk3YjQtMzNiODRkNzBjMTkxIiwiNjkxZDU4ODgtZTBkOC00OTIxLWJmNGEtYjFkNWIzNDZmNGVkIiwiMGQ2ZTdjOTAtNmJiOC00ZjQxLThjMDgtYjU4YTk5Mjg1MmY3IiwiNWEwMmM3OTEtMjkwMS00ZTdmLWIzZDYtYzY1NTBiOTE0NTY4IiwiZjQwYjYwOTYtZjJmMy00ZDA2LTg1MWYtZWYxOGJhN2VhZjE4IiwiMTU5NmU0YjQtZDA0ZS00NjNjLTkzOTEtMzIzZWYwNDlhMmYxIiwiMDJjMGM4YzQtMmVhMi00NjM3LTg0NDYtZTE4OThkMjI2NjM5IiwiZGFmMjBmZDEtMDc0OC00M2MxLWI4MzAtOWU1YmRjYzE2YmQyIiwiYWViZmIxZDQtZGNmMS00N2Q4LWEyM2EtNmI2YTE5N2M1YjkzIiwiYzNhMmRjZDctMjZmZi00N2FmLWFlNDktYzcxOWE4ZGM0YTZlIiwiZDIzNGNhZGUtY2Q5Yi00NjFhLTg4MmItOTM5NzFhZGYzODBlIiwiMTE5ZTk2ZTEtZWE3YS00MzA3LThiMWUtMmIwZTM4MDQ0ZjNhIiwiYWZhZjA3ZWEtYjY4Yy00OTVhLWFmZTgtMjc0YWJlOGZiZDNhIiwiNWVhZDM5ZjYtOWQ1My00NWFkLWJjNmItOTVmMjU2Njc2YjM0Il0sImlkdHlwIjoidXNlciIsImlwYWRkciI6IjM0Ljk4LjIwMy4xMTkiLCJuYW1lIjoiU3ViaHJhbmVlbCBDaGFrcmFib3J0eSIsIm9pZCI6IjhjOGVkYTNlLWQxMjAtNGMxOC04NGEyLTEwZjgwMmE4NThiYSIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0yNTc0MjE4OTQ0LTMxMjY3MTk0NjQtMTI3MzcwNDQ3MC05NTc0MCIsInB1aWQiOiIxMDAzMjAwMzMyQkNDMkFDIiwicmgiOiIwLkFUY0FBd2dUT1RnS3dVeWhGQkY3NXlWYzdaQWlNWDNJS0R4SG9PMk9VM1NiYlcwM0FFay4iLCJzY3AiOiJ1c2VyX2ltcGVyc29uYXRpb24iLCJzdWIiOiJXdW9YUF93Zm9DMTQyWGpDRWF1WHh2NXlMNkpRakNuQVY5ZnZPQmM2WFpVIiwidGlkIjoiMzkxMzA4MDMtMGEzOC00Y2MxLWExMTQtMTE3YmU3MjU1Y2VkIiwidW5pcXVlX25hbWUiOiJjaGFrc3ViQGFjdWl0eWtwLmNvbSIsInVwbiI6ImNoYWtzdWJAYWN1aXR5a3AuY29tIiwidXRpIjoiU3VYMjE2U2RzRXFlZUh4TFZuWnVBQSIsInZlciI6IjEuMCIsIndpZHMiOlsiYjc5ZmJmNGQtM2VmOS00Njg5LTgxNDMtNzZiMTk0ZTg1NTA5Il0sInhtc19pZHJlbCI6IjggMSJ9.Jbhy70zWlB8960YvfhjFGuCKFzSAjTPNxeBYGGPdTF5hVdP3FOdGRaSKQZiY4F_Esg4voQvSme2DYzkt6ippD5rQOvXi81k1SOJyqk61vvK38LWVT9yWfAntPC2eJpY64y2yPDW7jcuq69GqqtFCxbuFr6Mb5fD2ygqOU0Bjjd0sTK_ju7AMyT64YX0kt0_jGHeqkLjun_4EUkSFs5zav446RpiAOySkSSwhcvyZXCDC4F61duXrmDKMEOiBTKU9OAqjvFGRM-k4lfRA9nsBKn8GXAtkJC36xTdaKrvL9hi8PnN19lOT0g4eLJaLE6HcmFuBSEJK9ixaKdy-Tljiog"

In [12]:
# from openai import AzureOpenAI
# from opensearchpy import OpenSearch
# import json

# # Initialize Azure OpenAI client
# api_version = "2023-05-15"
# endpoint = "https://beat-sandbox-openai-00001.openai.azure.com/"

# # Ensure azure_token is properly initialized
# azure_token = azure_token  # Replace with your actual Azure AD token

# if not azure_token:
#     raise ValueError("Azure token must be provided")

# client = AzureOpenAI(
#     azure_deployment='text-embedding-ada-002',
#     api_version=api_version,
#     azure_endpoint=endpoint,
#     azure_ad_token=azure_token,
# )


# oai_client = AzureOpenAI(
#     azure_deployment='gpt-35-turbo-16k',
#     api_version=api_version,
#     azure_endpoint=endpoint,
#     azure_ad_token=azure_token,
# )

# class OpenSearchVectorStore:
#     def __init__(self, host, port, index_name, http_auth):
#         self.client = OpenSearch(
#             hosts=[{'host': host, 'port': port}],
#             http_auth=http_auth,
#             use_ssl=False,  # Disable SSL
#             verify_certs=False,  # Disable certificate verification
#             ssl_show_warn=False  # Suppress SSL warnings
#         )
#         self.index_name = index_name

#     def query(self, query_vector, n_results):
#         # Ensure query_vector is a list of floats
#         if not isinstance(query_vector, list) or not all(isinstance(i, float) for i in query_vector):
#             raise ValueError("query_vector must be a list of floats")

#         # Debugging: Print the query vector to ensure it's correctly formatted
#         print(f"Query Vector (type: {type(query_vector)}): {query_vector}")

#         query = {
#             "size": n_results,
#             "query": {
#                 "knn": {
#                     "field": "embedding",
#                     "query_vector": query_vector,
#                     "k": n_results,
#                     "num_candidates": 100
#                 }
#             }
#         }
#         # Debugging: Print the query JSON
#         query_json = json.dumps(query, indent=2)
#         print(f"Query JSON: {query_json}")

#         # Send the query as a dictionary, not as a JSON string
#         response = self.client.search(index=self.index_name, body=query)
#         return response

# def check_and_fix_embeddings(client, index_name):
#     # Retrieve all documents from the index
#     response = client.search(index=index_name, body={"query": {"match_all": {}}}, size=1000)
#     documents = response['hits']['hits']

#     for doc in documents:
#         doc_id = doc['_id']
#         source = doc['_source']
#         embedding = source.get('embedding', None)

#         # Debugging: Print document ID and embedding
#         print(f"Document ID: {doc_id}, Embedding: {embedding}")

#         # Check if the embedding field is correctly formatted
#         if not isinstance(embedding, list) or not all(isinstance(i, float) for i in embedding):
#             print(f"Document ID {doc_id} has an incorrectly formatted embedding: {embedding}")

#             # Fix the embedding field (example: convert to a list of floats)
#             # This is a placeholder; you need to implement the actual fix based on your data
#             fixed_embedding = [float(i) for i in embedding.split(',')] if isinstance(embedding, str) else []

#             # Update the document with the fixed embedding
#             client.update(
#                 index=index_name,
#                 id=doc_id,
#                 body={"doc": {"embedding": fixed_embedding}}
#             )
#             print(f"Document ID {doc_id} has been updated with fixed embedding: {fixed_embedding}")

# class RAG_from_scratch:
#     def __init__(self, vector_store):
#         self.vector_store = vector_store

#     def retrieve(self, query: str) -> list:
#         """
#         Retrieve relevant text from vector store.
#         """
#         # Convert the query to a vector using text-embedding-ada-002
#         query_vector = self.vectorize_query(query)
#         print(f"Query Vector: {query_vector}")  # Debugging: Print the query vector
#         results = self.vector_store.query(query_vector=query_vector, n_results=4)
#         return [doc['_source'] for doc in results['hits']['hits']]

#     def vectorize_query(self, query: str) -> list:
#         """
#         Convert the query string to a vector using text-embedding-ada-002.
#         """
#         response = client.embeddings.create(
#             model="text-embedding-ada-002",
#             input=query
#         )
#         if response.data and response.data[0] and response.data[0].embedding:
#             query_vector = response.data[0].embedding
#         else:
#             raise ValueError("Failed to retrieve embedding from response")
#         print(f"Query Vector: {query_vector}")  # Debugging: Print the query vector
#         return query_vector

#     def generate_completion(self, query: str, context_str: list) -> str:
#         """
#         Generate answer from context.
#         """
#         completion = (
#             oai_client.chat.completions.create(
#                 model="gpt-35-turbo-16k",
#                 temperature=0,
#                 messages=[
#                     {
#                         "role": "user",
#                         "content": f"We have provided context information below. \n"
#                         f"---------------------\n"
#                         f"{context_str}"
#                         f"\n---------------------\n"
#                         f"Given this information, please answer the question: {query}",
#                     }
#                 ],
#             )
#             .choices[0]
#             .message.content
#         )
#         if completion is None:
#             raise ValueError("Completion result is None")
#         return completion

#     def query(self, query: str) -> str:
#         context_str = self.retrieve(query)
#         completion = self.generate_completion(query, context_str)
#         if completion is None:
#             raise ValueError("Query result is None")
#         return completion

# # Initialize OpenSearch vector store with similar settings as os_client
# vector_store = OpenSearchVectorStore(
#     host='localhost',
#     port=9200,
#     index_name='embedded_texts',  # Replace with your actual index name
#     http_auth=('admin', 'H@thork!7210')
# )

# # Check and fix embeddings in the index
# check_and_fix_embeddings(vector_store.client, vector_store.index_name)

# # Initialize RAG_from_scratch object
# rag = RAG_from_scratch(vector_store=vector_store)

# # Example usage of the rag object
# query = "what is the capital of France?"
# response = rag.query(query)
# print(response)

In [13]:
from openai import AzureOpenAI
from opensearchpy import OpenSearch, RequestError
import json

# Initialize Azure OpenAI client
api_version = "2023-05-15"
endpoint = "https://beat-sandbox-openai-00001.openai.azure.com/"

# Ensure azure_token is properly initialized
azure_token = azure_token  # Replace with your actual Azure AD token

if not azure_token:
    raise ValueError("Azure token must be provided")

oai_client = AzureOpenAI(
    azure_deployment='gpt-35-turbo-16k',
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token=azure_token,
)

class OpenSearchVectorStore:
    def __init__(self, host, port, index_name, http_auth):
        self.client = OpenSearch(
            hosts=[{'host': host, 'port': port}],
            http_auth=http_auth,
            use_ssl=False,  # Disable SSL
            verify_certs=False,  # Disable certificate verification
            ssl_show_warn=False  # Suppress SSL warnings
        )
        self.index_name = index_name

    def query(self, query_vector, n_results):
        # Ensure query_vector is a list of floats
        if not isinstance(query_vector, list) or not all(isinstance(i, float) for i in query_vector):
            raise ValueError("query_vector must be a list of floats")

        # Debugging: Print the query vector to ensure it's correctly formatted
        print(f"Query Vector (type: {type(query_vector)}): {query_vector}")

        query = {
            "size": n_results,
            "query": {
                "knn": {
                    "field": "embedding",
                    "query_vector": query_vector,
                    "k": n_results,
                    "num_candidates": 100
                }
            }
        }
        # Debugging: Print the query JSON
        query_json = json.dumps(query, indent=2)
        print(f"Query JSON: {query_json}")

        try:
            # Send the query as a dictionary, not as a JSON string
            response = self.client.search(index=self.index_name, body=query)
            return response
        except RequestError as e:
            print(f"RequestError: {e.info}")
            # Handle the error gracefully, e.g., return an empty result
            return {"hits": {"hits": []}}

def check_and_fix_embeddings(client, index_name):
    # Retrieve all documents from the index
    response = client.search(index=index_name, body={"query": {"match_all": {}}}, size=1000)
    documents = response['hits']['hits']

    for doc in documents:
        doc_id = doc['_id']
        source = doc['_source']
        embedding = source.get('embedding', None)

        # Debugging: Print document ID and embedding
        print(f"Document ID: {doc_id}, Embedding: {embedding}")

        # Check if the embedding field is correctly formatted
        if not isinstance(embedding, list) or not all(isinstance(i, float) for i in embedding):
            print(f"Document ID {doc_id} has an incorrectly formatted embedding: {embedding}")

            # Fix the embedding field (example: convert to a list of floats)
            # This is a placeholder; you need to implement the actual fix based on your data
            fixed_embedding = [float(i) for i in embedding.split(',')] if isinstance(embedding, str) else []

            # Update the document with the fixed embedding
            client.update(
                index=index_name,
                id=doc_id,
                body={"doc": {"embedding": fixed_embedding}}
            )
            print(f"Document ID {doc_id} has been updated with fixed embedding: {fixed_embedding}")

class RAG_from_scratch:
    def __init__(self, vector_store):
        self.vector_store = vector_store

    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        # Convert the query to a vector using text-embedding-ada-002
        query_vector = self.vectorize_query(query)
        print(f"Query Vector: {query_vector}")  # Debugging: Print the query vector
        results = self.vector_store.query(query_vector=query_vector, n_results=4)
        return [doc['_source'] for doc in results['hits']['hits']]

    def vectorize_query(self, query: str) -> list:
        """
        Convert the query string to a vector using text-embedding-ada-002.
        """
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=query
        )
        if response.data and response.data[0] and response.data[0].embedding:
            query_vector = response.data[0].embedding
        else:
            raise ValueError("Failed to retrieve embedding from response")
        print(f"Query Vector: {query_vector}")  # Debugging: Print the query vector
        return query_vector

    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = (
            oai_client.chat.completions.create(
                model="gpt-35-turbo-16k",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"We have provided context information below. \n"
                        f"---------------------\n"
                        f"{context_str}"
                        f"\n---------------------\n"
                        f"Given this information, please answer the question: {query}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        if completion is None:
            raise ValueError("Completion result is None")
        return completion

    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        if completion is None:
            raise ValueError("Query result is None")
        return completion

# Initialize OpenSearch vector store with similar settings as os_client
vector_store = OpenSearchVectorStore(
    host='localhost',
    port=9200,
    index_name='embedded_texts',  # Replace with your actual index name
    http_auth=('admin', 'H@thork!7210')
)

# Check and fix embeddings in the index
check_and_fix_embeddings(vector_store.client, vector_store.index_name)

# Initialize RAG_from_scratch object
rag = RAG_from_scratch(vector_store=vector_store)

# Example usage of the rag object
query = "what is the capital of France?"
response = rag.query(query)
print(response)

Document ID: x0HWf5EB6PjdLD5D0hzs, Embedding: [0.004950651, -0.00999016, -0.016641805, -0.017314585, -0.023293449, -0.0046523428, -0.0052203983, 0.0044206777, -0.008092411, -0.022252543, -0.022544503, 0.00762908, 0.03437529, -0.015880166, -0.0009695025, -0.030008562, 0.013354064, -0.040290684, -0.0032226834, -0.015385101, -0.017949283, -0.009634729, 0.0033416895, -0.010802574, -0.00933642, 0.018825168, -0.001161499, -0.020640407, 0.029018432, -0.00887309, -0.0045285765, -0.0052743475, -0.04392116, -0.013493698, -0.014953505, -0.004236615, -0.013354064, 0.005429849, 0.0039097453, -0.0055885236, 0.011881563, -0.000360985, -0.00288788, -0.009844179, -0.016260985, 0.014306112, -0.0014193454, -0.027520543, -0.0070578516, 0.0062263957, 0.017124176, 0.012179871, -0.015550123, -0.016007105, -0.017466912, -0.028459897, -0.008796927, 0.018583983, -0.023153815, -0.009082541, -0.004512709, -0.005801148, -0.021592455, -0.011240517, -0.0034115063, -0.0091460105, -0.019586807, 0.016527558, -0.0231411

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [10]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\chaksub\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install numpy==1.26.4


  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapter-transformers 3.2.1.post0 requires huggingface-hub<0.14.0,>=0.11.0, but you have huggingface-hub 0.23.4 which is incompatible.
adapter-transformers 3.2.1.post0 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.19.1 which is incompatible.
trulens-eval 0.33.0 requires munch>=3.0.0, but you have munch 2.5.1.dev12 which is incompatible.

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip show numpy

Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

    * Redistributions of source code must retain the above copyright
       notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
       copyright notice, this list of conditions and the following
       disclaimer in the documentation and/or other materials provided
       with the distribution.

    * Neither the name of the NumPy Developers nor the names of any
       contributors may be used to endorse or promote products derived
       from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYR

In [14]:
import numpy as np
print(np.__version__)

1.26.4


In [15]:
!pip uninstall numpy -y

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


You can safely remove it manually.
You can safely remove it manually.


In [14]:
import os
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai.provider import AzureOpenAI

# Ensure azure_token is properly initialized
azure_token = azure_token  # Replace with your actual Azure AD token

if not azure_token:
    raise ValueError("Azure token must be provided")

os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://beat-sandbox-openai-00001.openai.azure.com/'
os.environ['AZURE_OPENAI_API_KEY'] = azure_token
os.environ['OPENAI_API_VERSION'] = '2023-05-15'


provider = AzureOpenAI(
    deployment_name='gpt-35-turbo-16k'
)


# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [15]:
from trulens.core import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_id="RAG v1",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],selector_nocheck=True,
)

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [16]:
with tru_rag as recording:
    rag.query(
        "Compare and contrast the historical significance, academic focus, and geographical context of the University of Washington and Washington State University. Additionally, discuss how the city of Seattle and the presence of major corporations like Starbucks influence the cultural and economic landscape of the region, particularly in relation to these universities."
    )

Query Vector: [0.018359487876296043, -0.016873674467206, 0.03480679914355278, -0.014896896667778492, -0.015930505469441414, -0.007280486635863781, -0.016395630314946175, 0.011156521737575531, -0.015529981814324856, -0.04483281075954437, -0.032041892409324646, -0.003649933496490121, 0.015323260799050331, -0.004237798973917961, 0.009935570880770683, -0.011860668659210205, 0.02863098308444023, -0.044677767902612686, -0.0026857696939259768, -0.01682199351489544, -0.03702905774116516, 0.00875984039157629, 0.004250718746334314, -0.01751968078315258, -0.011750848032534122, 0.036021288484334946, 0.01452221255749464, -0.01576254516839981, 0.001708685653284192, -0.013798686675727367, -0.014418852515518665, -0.0124162333086133, -0.030904922634363174, 0.016046786680817604, -0.005678392015397549, -0.010064772330224514, -0.005694542080163956, -0.010872279293835163, -0.004415450617671013, -0.000531743629835546, -0.018088165670633316, 0.011776687577366829, -0.009276645258069038, -0.006847662851214409,

## Check results

We can view results in the leaderboard.

In [17]:
tru.get_leaderboard()

,latency,total_cost
app_id,,


## Use guardrails

In addition to making informed iteration, we can also directly use feedback results as guardrails at inference time. In particular, here we show how to use the context relevance score as a guardrail to filter out irrelevant context before it gets passed to the LLM. This both reduces hallucination and improves efficiency.

To do so, we'll rebuild our RAG using the @context-filter decorator on the method we want to filter, and pass in the feedback function and threshold to use for guardrailing.

In [18]:
# from trulens.core.guardrails.base import context_filter

# # note: feedback function used for guardrail must only return a score, not also reasons
# f_context_relevance_score = Feedback(
#     provider.context_relevance, name="Context Relevance"
# )


# class filtered_RAG_from_scratch:
#     @instrument
#     @context_filter(f_context_relevance_score, 0.75, keyword_for_prompt="query")
#     def retrieve(self, query: str) -> list:
#         """
#         Retrieve relevant text from vector store.
#         """
#         results = vector_store.query(query_vector=query, n_results=4)
#         return [doc for sublist in results["documents"] for doc in sublist]

#     @instrument
#     def generate_completion(self, query: str, context_str: list) -> str:
#         """
#         Generate answer from context.
#         """
#         completion = (
#             oai_client.chat.completions.create(
#                 model="gpt-3.5-turbo",
#                 temperature=0,
#                 messages=[
#                     {
#                         "role": "user",
#                         "content": f"We have provided context information below. \n"
#                         f"---------------------\n"
#                         f"{context_str}"
#                         f"\n---------------------\n"
#                         f"Given this information, please answer the question: {query}",
#                     }
#                 ],
#             )
#             .choices[0]
#             .message.content
#         )
#         return completion

#     @instrument
#     def query(self, query: str) -> str:
#         context_str = self.retrieve(query=query)
#         completion = self.generate_completion(
#             query=query, context_str=context_str
#         )
#         return completion


# filtered_rag = filtered_RAG_from_scratch()

In [19]:
import ssl
import nltk

# Create an unverified SSL context
ssl._create_default_https_context = ssl._create_unverified_context

# Download the punkt tokenizer data
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chaksub\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
from trulens.core.guardrails.base import context_filter

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = Feedback(
    provider.context_relevance, name="Context Relevance"
)

class filtered_RAG_from_scratch:
    @instrument
    @context_filter(f_context_relevance_score, 0.75, keyword_for_prompt="query")
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        query_vector = self.vectorize_query(query)  # Convert query to vector
        results = vector_store.query(query_vector=query_vector, n_results=4)
        
        # Debugging: Print the results list to inspect its structure
        print(f"Results: {results}")
        
        # Assuming results is a list of documents or lists of documents
        documents = []
        for item in results:
            if isinstance(item, list):
                documents.extend(item)
            else:
                documents.append(item)
        
        return documents

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = (
            oai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"We have provided context information below. \n"
                        f"---------------------\n"
                        f"{context_str}"
                        f"\n---------------------\n"
                        f"Given this information, please answer the question: {query}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query=query)
        completion = self.generate_completion(
            query=query, context_str=context_str
        )
        return completion

    def vectorize_query(self, query: str) -> list:
        """
        Convert the query string to a vector using text-embedding-ada-002.
        """
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=query
        )
        if response.data and response.data[0] and response.data[0].embedding:
            query_vector = response.data[0].embedding
        else:
            raise ValueError("Failed to retrieve embedding from response")
        return query_vector

filtered_rag = filtered_RAG_from_scratch()

from trulens.core import TruCustomApp

filtered_tru_rag = TruCustomApp(
    filtered_rag,
    app_id="RAG v2",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

with filtered_tru_rag as recording:
    filtered_rag.query(
        query="Compare and contrast the historical significance, academic focus, and geographical context of the University of Washington and Washington State University. Additionally, discuss how the city of Seattle and the presence of major corporations like Starbucks influence the cultural and economic landscape of the region, particularly in relation to these universities."
    )

Query Vector (type: <class 'list'>): [0.018359487876296043, -0.016873674467206, 0.03480679914355278, -0.014896896667778492, -0.015930505469441414, -0.007280486635863781, -0.016395630314946175, 0.011156521737575531, -0.015529981814324856, -0.04483281075954437, -0.032041892409324646, -0.003649933496490121, 0.015323260799050331, -0.004237798973917961, 0.009935570880770683, -0.011860668659210205, 0.02863098308444023, -0.044677767902612686, -0.0026857696939259768, -0.01682199351489544, -0.03702905774116516, 0.00875984039157629, 0.004250718746334314, -0.01751968078315258, -0.011750848032534122, 0.036021288484334946, 0.01452221255749464, -0.01576254516839981, 0.001708685653284192, -0.013798686675727367, -0.014418852515518665, -0.0124162333086133, -0.030904922634363174, 0.016046786680817604, -0.005678392015397549, -0.010064772330224514, -0.005694542080163956, -0.010872279293835163, -0.004415450617671013, -0.000531743629835546, -0.018088165670633316, 0.011776687577366829, -0.009276645258069038,

## Record and operate as normal

In [21]:
# from trulens.core import TruCustomApp

# filtered_tru_rag = TruCustomApp(
#     filtered_rag,
#     app_id="RAG v2",
#     feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
# )

# with filtered_tru_rag as recording:
#     filtered_rag.query(
#         query="Compare and contrast the historical significance, academic focus, and geographical context of the University of Washington and Washington State University. Additionally, discuss how the city of Seattle and the presence of major corporations like Starbucks influence the cultural and economic landscape of the region, particularly in relation to these universities."
#     )

In [22]:
tru.get_leaderboard(app_ids=[])

,latency,total_cost
app_id,,
RAG v2,6.0,0.0


See the power of filtering!

In [23]:
from trulens.core.utils.trulens import get_feedback_result

last_record = recording.records[-1]
get_feedback_result(last_record, "Context Relevance")

,question,context,ret
0,Compare and contrast the historical significan...,hits,1.0


In [45]:
# Install the trulens package
!pip install trulens

# Verify the installation
#python -c "import trulens.dashboard; print('trulens.dashboard module is available')"


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
# import os

# # Define the path to the dashboard directory
# dashboard_dir = os.path.join(os.getcwd(), 'src', 'dashboard')

# # Check if the directory exists
# if os.path.exists(dashboard_dir):
#     # Define the path to the __init__.py file
#     init_file = os.path.join(dashboard_dir, '__init__.py')
    
#     # Create an empty __init__.py file if it doesn't exist
#     if not os.path.exists(init_file):
#         with open(init_file, 'w') as f:
#             pass
#     print(f"Created __init__.py in {dashboard_dir}")
# else:
#     print(f"Directory {dashboard_dir} does not exist")

Directory c:\Users\chaksub\OneDrive - acuitykp\Documents\GitHub\trulens_new\examples\quickstart\src\dashboard does not exist


In [55]:
# import os

# trulens_path = 'C:\\Users\\chaksub\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\trulens'
# print(os.listdir(trulens_path))

['core', 'feedback', 'nn', 'providers', 'utils', 'visualizations.py', '__init__.py', '__pycache__']


In [57]:
# import os

# trulens_path = 'C:\\Users\\chaksub\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\trulens'

# # Create the 'dashboard' directory if it doesn't exist
# dashboard_path = os.path.join(trulens_path, 'dashboard')
# if not os.path.exists(dashboard_path):
#     os.makedirs(dashboard_path)

# # Verify the directory creation
# print(os.listdir(trulens_path))

['core', 'dashboard', 'feedback', 'nn', 'providers', 'utils', 'visualizations.py', '__init__.py', '__pycache__']


In [61]:
# import os

# trulens_path = 'C:\\Users\\chaksub\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\trulens'
# dashboard_path = os.path.join(trulens_path, 'dashboard')

# # Create the directory structure
# structure = {
#     'components': {
#         'record_viewer': {}
#     },
#     'pages': {},
#     'ux': {}
# }

# def create_structure(base_path, structure):
#     for name, sub_structure in structure.items():
#         path = os.path.join(base_path, name)
#         os.makedirs(path, exist_ok=True)
#         create_structure(path, sub_structure)

# create_structure(dashboard_path, structure)

# # Verify the directory creation
# print("Structure of dashboard directory:")
# for root, dirs, files in os.walk(dashboard_path):
#     level = root.replace(dashboard_path, '').count(os.sep)
#     indent = ' ' * 4 * (level)
#     print(f"{indent}{os.path.basename(root)}/")
#     subindent = ' ' * 4 * (level + 1)
#     for f in files:
#         print(f"{subindent}{f}")

Structure of dashboard directory:
dashboard/
    components/
        record_viewer/
    pages/
    ux/


In [62]:
# import os
# import shutil

# trulens_path = 'C:\\Users\\chaksub\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\trulens'
# dashboard_path = os.path.join(trulens_path, 'dashboard')

# # Create the 'dashboard' directory if it doesn't exist
# if not os.path.exists(dashboard_path):
#     os.makedirs(dashboard_path)

# # Define the source path
# src_dashboard_path = 'C:\\Users\\chaksub\\OneDrive - acuitykp\\Documents\\GitHub\\trulens_new\\src\\dashboard'

# # Function to copy the contents of the source directory to the destination directory
# def copy_directory_contents(src, dest):
#     for item in os.listdir(src):
#         s = os.path.join(src, item)
#         d = os.path.join(dest, item)
#         if os.path.isdir(s):
#             shutil.copytree(s, d, dirs_exist_ok=True)
#         else:
#             shutil.copy2(s, d)

# # Copy the contents
# copy_directory_contents(src_dashboard_path, dashboard_path)

# # Verify the copy operation
# print("Contents of the destination dashboard directory:")
# for root, dirs, files in os.walk(dashboard_path):
#     level = root.replace(dashboard_path, '').count(os.sep)
#     indent = ' ' * 4 * (level)
#     print(f"{indent}{os.path.basename(root)}/")
#     subindent = ' ' * 4 * (level + 1)
#     for f in files:
#         print(f"{subindent}{f}")

Contents of the destination dashboard directory:
dashboard/
    pyproject.toml
    README.md
    setup.py
    components/
        record_viewer/
    pages/
    react_components/
        .gitignore
        record_viewer/
            .eslintrc.cjs
            .gitignore
            .prettierrc
            index.html
            package-lock.json
            package.json
            README.md
            tsconfig.json
            tsconfig.node.json
            vite.config.ts
            src/
                main.tsx
                react-app-env.d.ts
                RecordInfo.tsx
                RecordViewer.tsx
                vite-env.d.ts
                assets/
                    fonts.css
                    SourceCodePro-Bold.ttf
                    SourceCodePro-Regular.ttf
                    SourceCodePro-SemiBold.ttf
                JSONViewer/
                    index.ts
                    JSONViewer.tsx
                LabelAndValue/
                    index.ts
          

In [64]:
# import os

# def find_function_definition(directory, function_name):
#     for root, _, files in os.walk(directory):
#         for file in files:
#             if file.endswith(".py"):
#                 file_path = os.path.join(root, file)
#                 with open(file_path, 'r') as f:
#                     for line in f:
#                         if line.strip().startswith(f"def {function_name}"):
#                             print(f"Found {function_name} in {file_path}")
#                             return file_path
#     print(f"{function_name} not found in {directory}")
#     return None

# dashboard_path = 'C:\\Users\\chaksub\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\trulens\\dashboard'
# function_name = "run_dashboard"
# find_function_definition(dashboard_path, function_name)

Found run_dashboard in C:\Users\chaksub\AppData\Local\Programs\Python\Python311\Lib\site-packages\trulens\dashboard\trulens\dashboard\run.py


'C:\\Users\\chaksub\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\trulens\\dashboard\\trulens\\dashboard\\run.py'

In [35]:
import trulens

#import trulens.dashboard.trulens.dashboard
help(trulens.dashboard.trulens)

Help on package trulens.dashboard.trulens in trulens.dashboard:

NAME
    trulens.dashboard.trulens

PACKAGE CONTENTS
    dashboard (package)

FILE
    (built-in)




In [60]:
from trulens.dashboard.trulens.dashboard.run import run_dashboard
# Assuming 'tru' is defined somewhere in your code
# Replace 'tru' with the actual object if needed
run_dashboard(tru)

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.39.48.203:58413



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [57]:
import sys
import os

# Add the src directory to the Python path
src_path = r'C:\Users\chaksub\OneDrive - acuitykp\Documents\GitHub\trulens_new\src'
sys.path.append(src_path)

# List the contents of the trulens/dashboard directory
dashboard_path = os.path.join(src_path, 'dashboard', 'trulens', 'dashboard')
print(os.listdir(dashboard_path))

# Correct import statement
from dashboard.trulens.dashboard.streamlit_utils import init_from_args

['appui.py', 'components', 'display.py', 'Leaderboard.py', 'notebook_utils.py', 'pages', 'py.typed', 'run.py', 'streamlit.py', 'streamlit_utils.py', 'ux', '__init__.py', '__pycache__']


PackageNotFoundError: No package metadata was found for dashboard.trulens.dashboard

In [59]:
import sys
import os

# Add the src directory to the Python path
src_path = r'C:\Users\chaksub\OneDrive - acuitykp\Documents\GitHub\trulens_new\src'
sys.path.append(src_path)

# List the contents of the trulens/dashboard directory
dashboard_path = os.path.join(src_path, 'dashboard', 'trulens', 'dashboard')
print(os.listdir(dashboard_path))

# Correct import statement
from trulens.dashboard.trulens.dashboard.streamlit_utils import init_from_args

['appui.py', 'components', 'display.py', 'Leaderboard.py', 'notebook_utils.py', 'pages', 'py.typed', 'run.py', 'streamlit.py', 'streamlit_utils.py', 'ux', '__init__.py', '__pycache__']


PackageNotFoundError: No package metadata was found for trulens.dashboard.trulens.dashboard

In [41]:
import os

# List the contents of the trulens/dashboard directory
dashboard_path = r'C:\Users\chaksub\OneDrive - acuitykp\Documents\GitHub\trulens_new\src\dashboard\trulens\dashboard'
print(os.listdir(dashboard_path))

['appui.py', 'components', 'display.py', 'Leaderboard.py', 'notebook_utils.py', 'pages', 'py.typed', 'run.py', 'streamlit.py', 'streamlit_utils.py', 'ux', '__init__.py']


In [49]:
!pip install --upgrade trulens


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
print(np.__version__)

1.26.4


In [38]:
import os

def find_file(filename, search_path):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# Replace 'your_project_directory' with the path to your project directory
project_directory = 'C:\\Users\\chaksub\\OneDrive - acuitykp\\Documents\\GitHub\\trulens_new'
file_to_find = 'Leaderboard.py'

file_path = find_file(file_to_find, project_directory)

if file_path:
    print(f"File found: {file_path}")
else:
    print("File not found")

File found: C:\Users\chaksub\OneDrive - acuitykp\Documents\GitHub\trulens_new\src\dashboard\trulens\dashboard\Leaderboard.py
